In [ ]:
#!pip install --upgrade tensorflow==1.15

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.layers import SimpleRNNCell, RNN, Dense
from sklearn.model_selection import train_test_split

In [ ]:
def load_dataset():
  df = pd.read_csv('classification.csv')
  cap_shape_temp = []
  odor_temp = []
  habitat_temp = []
  df = df[['class', 'cap-shape', 'cap-color', 'odor', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color', 'ring-number', 'habitat']]

  for item in df['cap-shape']:
    if item == 'b':
      cap_shape_temp.append(1)
    elif item == 'c':
      cap_shape_temp.append(2)
    elif item == 'x':
      cap_shape_temp.append(3)
    elif item == 'f':
      cap_shape_temp.append(4)
    elif item == 'k':
      cap_shape_temp.append(5)
    elif item == 's':
      cap_shape_temp.append(6)
  df['cap-shape'] = cap_shape_temp

  for item in df['odor']:
    if item == 'a':
      odor_temp.append(1)
    elif item == 'l':
      odor_temp.append(2)
    elif item == 'c':
      odor_temp.append(3)
    elif item == 'y':
      odor_temp.append(4)
    elif item == 'f':
      odor_temp.append(5)
    elif item == 'm':
      odor_temp.append(6)
    elif item == 'n':
      odor_temp.append(7)
    elif item == 'p':
      odor_temp.append(8)
    elif item == 's':
      odor_temp.append(9)
  df['odor'] = odor_temp

  for item in df['habitat']:
    if item == 'g':
      habitat_temp.append(1)
    elif item == 'l':
      habitat_temp.append(2)
    elif item == 'm':
      habitat_temp.append(3)
    elif item == 'p':
      habitat_temp.append(4)
    elif item == 'u':
      habitat_temp.append(5)
    elif item == 'w':
      habitat_temp.append(6)
    elif item == 'd':
      habitat_temp.append(7)
  df['habitat'] = habitat_temp

  return df

In [ ]:
df = load_dataset()
features = df[['cap-shape', 'cap-color', 'odor', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color', 'ring-number', 'habitat']]
#df[['class']] = LabelEncoder().fit_transform(df[['class']])
label = df[['class']]

In [ ]:
normalize_data = StandardScaler().fit_transform(features)
mean = tf.reduce_mean(normalize_data, axis=0)
data_centered = normalize_data-mean

In [ ]:
from sklearn.decomposition import PCA


with tf.Session() as sess:
  df = sess.run(data_centered)
  pca = PCA(n_components = 4)
  pca_fit = pca.fit(features)
  result = pca_fit.transform(features)
  

In [ ]:
label

,class
0,p
1,e
2,e
3,p
4,e
...,...
8119,e
8120,e
8121,e
8122,p


In [ ]:
encoder = OneHotEncoder(handle_unknown = 'ignore').fit(label)
label = encoder.transform(label).toarray()

In [ ]:
split = int(len(result) * 0.7)
split2 = int(len(result)* 0.2)
rand_train = np.random.randint(len(result), size=split)
rand_test = np.random.randint(len(result), size=split2)

x_train = result[rand_train]
x_test = result[rand_test]
y_train = label[rand_train]
y_test = label[rand_test]
label

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [ ]:
n_features = x_train.shape[1]
n_classes = y_train.shape[1]

In [ ]:
neurons = [8, 16, 16, 16, 8, n_classes]
n_layers = len(neurons)
timestep = 12

In [ ]:
input_tensor = tf.placeholder(tf.float32)
label_tensor = tf.placeholder(tf.float32)

In [ ]:
def forward():
  a = input_tensor
  for i in range(n_layers):
    w = parameters[f'W{i+1}']
    b = parameters[f'B{i+1}']

    z = tf.matmul(a, w) + b
    if i == n_layers - 1:
      a = tf.nn.softmax(z)
    else:
      a = tf.nn.tanh(z)
  return a

In [ ]:
parameters = {}
for i in range(n_layers):
  parameters[f'W{i+1}'] = tf.Variable(tf.random.normal([n_features if (i == 0) else neurons[i-1], neurons[i]]),tf.float32)
  parameters[f'B{i+1}'] = tf.Variable(tf.random.normal([1, neurons[i]]),tf.float32)

In [ ]:
saver = tf.train.Saver()

In [ ]:
y_predict = forward()
EPOCH = 2500
ALPHA = 0.5

In [ ]:
with tf.Session() as sess:
  logits_tensor = forward()
  loss_tensor = tf.reduce_mean(0.5 * (label_tensor - logits_tensor) ** 2)

  optimizer = tf.train.GradientDescentOptimizer(ALPHA).minimize(loss_tensor)

  sess.run(tf.global_variables_initializer())

  best_loss = float('inf')
  for i in range(1, EPOCH+1):
    sess.run(optimizer, feed_dict={
        input_tensor:x_train,
        label_tensor:y_train
    })
    loss = sess.run(loss_tensor, feed_dict={
        input_tensor:x_train,
        label_tensor:y_train
    })
    if i % 25 == 0:
      print(f'Epoch {i} Loss {loss:.4f} ')
    y_pred=sess.run(y_predict, feed_dict={
            input_tensor: x_train,
            label_tensor: y_train
        })
    y_true = np.argmax(y_train,1)
    y_preds = np.argmax(y_pred,1)

    if i == 125:
        val_loss = sess.run(loss_tensor, feed_dict={
        input_tensor:x_test,
        label_tensor:y_test
        })
        best_loss = val_loss
        saver.save(sess, './best_model.ckpt')
    
    if i % 125 == 0:
      saver.restore(sess, './best_model.ckpt')
      if val_loss < best_loss:
        best_loss = val_loss
        saver.save(sess, './best_model.ckpt')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 25 Loss 0.0547 
Epoch 50 Loss 0.0463 
Epoch 75 Loss 0.0405 
Epoch 100 Loss 0.0385 
Epoch 125 Loss 0.0369 
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
tf.float32 has type DType, but expected one of: int, long, bool
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
tf.float32 has type DType, but expected one of: int, long, bool
INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Epoch 150 Loss 0.0354 
Epoch 175 Loss 0.0337 
Epoch 200 Loss 0.0335 
Epoch 225 Loss 0.0314 
Epoch 250 Loss 0.0302 
INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Epoch 275 Loss 0.0354 
Epoch 300 Loss 0.0337 
Epoch 325 Loss 0.0335 
Epoch 350 Loss 0.0314 
Epoch 375 Loss 0.0302 
INFO:tensorflow:Restoring parameters

In [ ]:
split3 = int(len(result) * 0.1)
rand_train = np.random.randint(len(result), size=split)
dataacc = result[rand_train]

split2 = int(len(dataacc)* 0.2)
rand_test = np.random.randint(len(dataacc), size=split2)

x_test = result[rand_test]
y_test = label[rand_test]

with tf.Session() as sess:
  logits_tensor = forward()
  loss_tensor = tf.reduce_mean(0.5 * (label_tensor - logits_tensor) ** 2)
  true_preds_tensor = tf.equal(tf.argmax(logits_tensor, axis=1), tf.argmax(label_tensor, axis=1))
  acc_tensor = tf.reduce_mean(tf.cast(true_preds_tensor, tf.float32))

  optimizer = tf.train.GradientDescentOptimizer(ALPHA).minimize(loss_tensor)

  sess.run(tf.global_variables_initializer())

  for i in range(1, EPOCH+1):
    sess.run(optimizer, feed_dict={
        input_tensor:x_train,
        label_tensor:y_train
    })
    acc = sess.run(acc_tensor, feed_dict={
        input_tensor:x_train,
        label_tensor:y_train
    })
    if i % 2500 == 0:
      print(f'Accuracy {(acc*100):.4f} % ')
    y_pred=sess.run(y_predict, feed_dict={
            input_tensor: x_train,
            label_tensor: y_train
        })
    y_true = np.argmax(y_train,1)
    y_preds = np.argmax(y_pred,1)

Accuracy 99.7010 % 
